In [2]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('OpenRA_FileCommits.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)

df['fullpath'] = df['new_path']
df.drop('project_name', axis=1, inplace=True)
df.drop('project_path', axis=1, inplace=True)
df.drop('new_path', axis=1, inplace=True)
df.drop('old_path', axis=1, inplace=True)

df.head()

,hash,message,author_name,author_date,in_main,is_merge,num_deletes,num_inserts,net_lines,num_files,branches,filename,parents,fullpath
0,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\n\ngit-svn-id: svn://svn....,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,Blowfish.cs,NaN,MixBrowser\Blowfish.cs
1,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\n\ngit-svn-id: svn://svn....,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MM.DAT,NaN,MixBrowser\MM.DAT
2,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\n\ngit-svn-id: svn://svn....,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MixBrowser.csproj,NaN,MixBrowser\MixBrowser.csproj
3,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\n\ngit-svn-id: svn://svn....,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MixEntry.cs,NaN,MixBrowser\MixEntry.cs
4,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\n\ngit-svn-id: svn://svn....,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,Program.cs,NaN,MixBrowser\Program.cs


In [3]:
df_files = pd.read_csv('filesizes.csv')

df_files.drop('Unnamed: 0', axis=1, inplace=True)

def fix_file_path(row):
    if row['path'] == '.':
        row['fullpath'] = row['project'] + '\\' + row['filename']
    else:
        row['fullpath'] = row['project'] + '\\' + row['path'] + '\\' + row['filename']
    return row

df_files = df_files.apply(fix_file_path, axis=1)

df_files.head()

,fullpath,project,path,filename,ext,lines
0,OpenRA.Game\Activities\Activity.cs,OpenRA.Game,Activities,Activity.cs,.cs,291
1,OpenRA.Game\Activities\CallFunc.cs,OpenRA.Game,Activities,CallFunc.cs,.cs,33
2,OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,.cs,645
3,OpenRA.Game\CPos.cs,OpenRA.Game,.,CPos.cs,.cs,150
4,OpenRA.Game\CryptoUtil.cs,OpenRA.Game,.,CryptoUtil.cs,.cs,261


In [4]:
df_merged = pd.merge(df, df_files, left_on='fullpath', right_on='fullpath')

df_merged.head()

,hash,message,author_name,author_date,in_main,is_merge,num_deletes,num_inserts,net_lines,num_files,branches,filename_x,parents,fullpath,project,path,filename_y,ext,lines
0,7881deca304e27f9cb20304c81731433954bec4f,"Everything is now OpenRA, not OpenRa",alzeih,2010-02-27 21:10:22+13:00,True,False,21,26919,26898,299,bleed,Actor.cs,11b926a42295e232676383c8909a0a1bcb0a101d,OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,.cs,645
1,8e5a5552c4fc078a44672294ed14570375037f78,Killed a bunch of logging statements,alzeih,2010-03-18 19:50:11+13:00,True,False,17,1,-16,10,bleed,Actor.cs,ec544489945ed0e895cf618583e2190f4e72f386,OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,.cs,645
2,faa8f54e7cd71ef3dcc4c526baa8b6639e1a49ed,"impose the hack on WALLs, not everything-else",Chris Forbes,2010-03-27 10:05:01+13:00,True,False,41,33,-8,6,bleed,Actor.cs,dd6f61a29f8cc3059231f3d22470c664c151d819,OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,.cs,645
3,13e6711a3b679a8522de76d2724c70798b3f40cb,further unhacking to localise extended damage ...,Chris Forbes,2010-03-27 10:11:24+13:00,True,False,27,27,0,4,bleed,Actor.cs,faa8f54e7cd71ef3dcc4c526baa8b6639e1a49ed,OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,.cs,645
4,d2ad90aa5b54e4bdb87b0f5215ba046bd0f2af2d,"saner planes, part 1",Chris Forbes,2010-03-27 00:23:24+13:00,True,False,2,54,52,4,bleed,Actor.cs,092c9bede0e15099e6583c532fb1c536f15419e7,OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,.cs,645


In [5]:
# Define common variables for plotly
from Theming import theme, template

In [13]:
df_grouped = df_merged.groupby(['fullpath', 'project', 'path', 'filename_y']).agg(
    avg_net=('net_lines','mean'),
    commits=('author_date', 'count')
)
df_grouped.head()

,,,,avg_net,commits
fullpath,project,path,filename_y,,
OpenRA.Game\Activities\Activity.cs,OpenRA.Game,Activities,Activity.cs,21.125000,48
OpenRA.Game\Activities\CallFunc.cs,OpenRA.Game,Activities,CallFunc.cs,59.928571,14
OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,133.613953,215
OpenRA.Game\CPos.cs,OpenRA.Game,.,CPos.cs,32.928571,42
OpenRA.Game\CVec.cs,OpenRA.Game,.,CVec.cs,64.678571,28


In [14]:
# Histogram of lines of code
fig = px.histogram(df_grouped,
                   x="avg_net",
                   color=df_grouped.index.get_level_values(1),
                   title='Average Net Lines',
                   labels={'lines': 'Lines of Code'},
                   template=template,
                   color_discrete_sequence=theme)
fig.show()

In [15]:
# Overall box plot for all code
fig = px.box(df_grouped,
             title='Distribution of Lines of Code',
             x='avg_net',
             color=df_grouped.index.get_level_values(1),
             height=275,
             template=template,
             color_discrete_sequence=theme,
             #hover_data=['project','path','filename'],
             labels={
                 'project': 'Project',
                 'lines': 'Lines of Code',
             },
             points='suspectedoutliers')
fig.update_traces(quartilemethod='linear', jitter=1)
fig.show()

In [16]:
# Box plot for distribution by projects
fig = px.box(df_grouped,
             x='avg_net',
             y=df_grouped.index.get_level_values(1),
#             hover_data=['path','filename'],
             labels={
               'project': 'Project',
               'lines': 'Lines of Code',
             },
             title='Distribution of Lines of Code by Project',
             points='suspectedoutliers',
             template=template,
             color_discrete_sequence=theme,
             color=df_grouped.index.get_level_values(1))
fig.update_traces(quartilemethod='linear', jitter=0.5)
fig.update_layout(showlegend=False)
fig.update_yaxes(tickangle=30, tickfont={'size': 10})
fig.show()

In [23]:
# Box plot for commits by projects
fig = px.box(df_grouped,
             x='commits',
             y=df_grouped.index.get_level_values(1),
#             hover_data=['path','filename'],
             labels={
               'project': 'Project',
               'lines': 'Lines of Code',
             },
             title='Commits by File by Project',
             points='suspectedoutliers',
             template=template,
             color_discrete_sequence=theme,
             color=df_grouped.index.get_level_values(1))
fig.update_traces(quartilemethod='linear', jitter=0.5)
fig.update_layout(showlegend=False)
fig.update_yaxes(tickangle=30, tickfont={'size': 10})
fig.show()

In [21]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='commits',
                 title='Commits by Area of Code',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 template=template,
                 height=800,
                 values='commits')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [25]:
# Sunburst diagram. Same data as a treemap, but different presentation
fig = px.sunburst(df_grouped,
                 path=[df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='commits',
                 title='Files receiving frequent commits',
                 #hover_data=['fullpath'],
                 color_continuous_scale='sunsetdark',
                 template=template,
                 width=1024,
                 height=800,
                 values='commits')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()